# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://haqnawaz.org")
ed.links

['#introduction',
 '#imam-budair',
 '#imam-juhany',
 '#imam-ghamidi',
 '#isdp',
 '#altasreef',
 '#easyquran',
 '#mohaddis',
 '#tanzeem',
 '#ai-nlp-expertise',
 '#The-Noble-Quran',
 '#Tazkeer-ul-Quran',
 '#seerat-quiz',
 '#tuheed',
 '#publications',
 '#brainiac',
 '#nascon',
 '#contact',
 '#introduction',
 '#contact',
 'https://jamiaashrafia.org/',
 'https://ili.digital/',
 'https://www.easyquranwahadees.com/',
 'https://tanzeemdigitallibrary.com/',
 'https://altasreef.vercel.app',
 'https://play.google.com/store/apps/details?id=com.maktaba.maariful.quran',
 'https://play.google.com/store/apps/details?id=com.asantarjuma',
 'https://play.google.com/store/apps/details?id=tazkeerul.quran',
 'https://play.google.com/store/apps/details?id=com.anfzquizapp',
 'https://play.google.com/store/apps/details?id=com.tuheed',
 'https://isdp.info',
 'https://isdp.info/students-in-field.php',
 'https://altasreef.vercel.app',
 'https://altasreef.vercel.app/exam',
 'https://aclanthology.org/2025.abjadnlp-

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
link_system_prompt = (
    "You are provided with a list of links found on a personal portfolio website. "
    "Your task is to identify the links most relevant for inclusion in a personal profile brochure, "
    "such as links to About, Projects, AI & NLP, Mobile Apps, Publications, Awards, or Contact sections.\n"
    "Respond in JSON format as shown below:\n"
    "{\n"
    '    "links": [\n'
    '        {"type": "about section", "url": "https://haqnawaz.org#about-me"},\n'
    '        {"type": "projects section", "url": "https://haqnawaz.org#projects"},\n'
    '        {"type": "AI & NLP section", "url": "https://haqnawaz.org#ai-nlp"},\n'
    '        {"type": "mobile apps section", "url": "https://haqnawaz.org#mobile-apps"},\n'
    '        {"type": "publications section", "url": "https://haqnawaz.org#publications"},\n'
    '        {"type": "awards section", "url": "https://haqnawaz.org#awards"},\n'
    '        {"type": "contact section", "url": "https://haqnawaz.org#contact"}\n'
    '    ]\n'
    "}"
)


In [7]:
print(link_system_prompt)

You are provided with a list of links found on a personal portfolio website. Your task is to identify the links most relevant for inclusion in a personal profile brochure, such as links to About, Projects, AI & NLP, Mobile Apps, Publications, Awards, or Contact sections.
Respond in JSON format as shown below:
{
    "links": [
        {"type": "about section", "url": "https://haqnawaz.org#about-me"},
        {"type": "projects section", "url": "https://haqnawaz.org#projects"},
        {"type": "AI & NLP section", "url": "https://haqnawaz.org#ai-nlp"},
        {"type": "mobile apps section", "url": "https://haqnawaz.org#mobile-apps"},
        {"type": "publications section", "url": "https://haqnawaz.org#publications"},
        {"type": "awards section", "url": "https://haqnawaz.org#awards"},
        {"type": "contact section", "url": "https://haqnawaz.org#contact"}
    ]
}


In [8]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [9]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://haqnawaz.org - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
#introduction
#imam-budair
#imam-juhany
#imam-ghamidi
#isdp
#altasreef
#easyquran
#mohaddis
#tanzeem
#ai-nlp-expertise
#The-Noble-Quran
#Tazkeer-ul-Quran
#seerat-quiz
#tuheed
#publications
#brainiac
#nascon
#contact
#introduction
#contact
https://jamiaashrafia.org/
https://ili.digital/
https://www.easyquranwahadees.com/
https://tanzeemdigitallibrary.com/
https://altasreef.vercel.app
https://play.google.com/store/apps/details?id=com.maktaba.maariful.quran
https://play.google.com/store/apps/details?id=com.asantarjuma
https://play.google.com/store/apps/details?id=tazkeerul.quran
https://play.google.com/store/apps/details?id=com.anfzquizapp
https://play.google.com/store/apps/details?id=com.tuheed
http

In [10]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [11]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/spaces',
 '/models',
 '/Qwen/Qwen2.5-Omni-7B',
 '/deepseek-ai/DeepSeek-V3-0324',
 '/all-hands/openhands-lm-32b-v0.1',
 '/meta-llama/Llama-4-Scout-17B-16E-Instruct',
 '/openfree/flux-chatgpt-ghibli-lora',
 '/models',
 '/spaces/enzostvs/deepsite',
 '/spaces/jamesliu1217/EasyControl_Ghibli',
 '/spaces/VAST-AI/TripoSG',
 '/spaces/ByteDance/InfiniteYou-FLUX',
 '/spaces/Stable-X/Hi3DGen',
 '/spaces',
 '/datasets/nvidia/Llama-Nemotron-Post-Training-Dataset-v1',
 '/datasets/virtuoussy/Multi-subject-RLVR',
 '/datasets/glaiveai/reasoning-v1-20m',
 '/datasets/open-thoughts/OpenThoughts2-1M',
 '/datasets/FreedomIntelligence/medical-o1-reasoning-SFT',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/g

In [12]:
get_links("https://huggingface.co")

{'links': [{'type': 'home', 'url': 'https://huggingface.co/'},
  {'type': 'models section', 'url': 'https://huggingface.co/models'},
  {'type': 'datasets section', 'url': 'https://huggingface.co/datasets'},
  {'type': 'spaces section', 'url': 'https://huggingface.co/spaces'},
  {'type': 'posts section', 'url': 'https://huggingface.co/posts'},
  {'type': 'documentation', 'url': 'https://huggingface.co/docs'},
  {'type': 'enterprise', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog', 'url': 'https://huggingface.co/blog'},
  {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'},
  {'type': 'status', 'url': 'https://status.huggingface.co/'},
  {'type': 'GitHub', 'url': 'https://github.com/huggingface'},
  {'type': 'Twitter', 'url': 'https://twitter.com/huggingface'},
  {'type': 'LinkedIn',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [13]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [14]:
# print(get_all_details("https://huggingface.co"))

In [15]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [16]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [ ]:
# get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'homepage', 'url': 'https://huggingface.co/'}, {'type': 'models section', 'url': 'https://huggingface.co/models'}, {'type': 'datasets section', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces section', 'url': 'https://huggingface.co/spaces'}, {'type': 'posts section', 'url': 'https://huggingface.co/posts'}, {'type': 'documentation section', 'url': 'https://huggingface.co/docs'}, {'type': 'enterprise section', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing section', 'url': 'https://huggingface.co/pricing'}, {'type': 'blog section', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub repository', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter profile', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn profile', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [ ]:
create_brochure("HuggingFace", "https://huggingface.co")

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [18]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("HuggingFace", "https://haqnawaz.org")

Found links: {'links': [{'type': 'about section', 'url': 'https://haqnawaz.org#introduction'}, {'type': 'projects section', 'url': 'https://haqnawaz.org#projects'}, {'type': 'AI & NLP section', 'url': 'https://haqnawaz.org#ai-nlp-expertise'}, {'type': 'mobile apps section', 'url': 'https://play.google.com/store/apps/details?id=com.maktaba.maariful.quran'}, {'type': 'mobile apps section', 'url': 'https://play.google.com/store/apps/details?id=tazkeerul.quran'}, {'type': 'mobile apps section', 'url': 'https://play.google.com/store/apps/details?id=com.anfzquizapp'}, {'type': 'mobile apps section', 'url': 'https://play.google.com/store/apps/details?id=com.tuheed'}, {'type': 'publications section', 'url': 'https://aclanthology.org/2025.abjadnlp-1.14.pdf'}, {'type': 'publications section', 'url': 'https://link.springer.com/article/10.1007/s10462-022-10313-2'}, {'type': 'publications section', 'url': 'https://aclanthology.org/2022.osact-1.18/'}, {'type': 'publications section', 'url': 'https:/

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


# HuggingFace Brochure

## Bridging AI & Islamic Scholarship

### About Us
HuggingFace, spearheaded by *Haq Nawaz*, is at the forefront of merging modern artificial intelligence (AI) with classical Islamic scholarship. With a robust background in Natural Language Processing (NLP), Haq Nawaz is not just a faculty member at Jamia Ashrafia Lahore but is also a dedicated PhD scholar and data science consultant. Our mission is to innovate and digitize Islamic education through award-winning software and applications, enhancing accessibility and engagement with religious texts.

### Our Vision
We are committed to pioneering efforts in Islamic software development, creating tools that benefit education, research, and broader public outreach. Our projects aim to make classical Islamic literature, the Qur'an, and Hadith easily searchable and interactable, ensuring that this vital knowledge is accessible to everyone.

### Notable Projects
- **Easy Quran**: An interactive platform that simplifies access to Quranic texts.
- **Mohaddis**: A tailored solution for Hadith studies.
- **Tanzeem**: A digital library assembling Islamic references.
- **Altasreef**: An award-winning platform for Arabic verb conjugation that earned 1st prize at UET Lahore’s Brainiac 2018.
- **Mobile Applications**:
  - **The Noble Quran**: Offering esteemed translations and tafsir.
  - **Tazkeer Ul Quran**: Multilingual thematic analysis.
  - **Seerat Quiz**: Educational interactive platform for children.

### Recognition
HuggingFace's contributions have been nationally recognized at events like NaSCon 2019 for innovative research and development in Quranic NLP annotation. We pride ourselves on our role in the wider scholarly community, having hosted discussions with esteemed Islamic leaders regarding the integration of technology and traditional scholarship.

### Company Culture
At HuggingFace, we foster an environment that encourages collaboration, innovation, and a deep respect for the principles of Islamic scholarship. Our culture thrives on bridging traditional and modern methodologies, promoting continuous learning, and empowering our employees to contribute to meaningful projects that have a tangible impact on society.

### Career Opportunities
We are always on the lookout for passionate individuals who desire to contribute to the intersection of AI and Islamic scholarship. Our **Intensive Software Development Program (ISDP)** equips recent graduates of Islamic madrasas with essential computer science and software engineering skills, preparing them for rewarding careers in technology.

Join us in our mission to inspire future generations by advancing Islamic knowledge through technology. Together, we can redefine how classical learning is approached in the digital age.

## Get in Touch
For inquiries related to partnerships, careers, or projects, contact us at:
- Email: [info@huggingface.com](mailto:info@huggingface.com)
- Phone: +123-456-7890

### Follow Us
Stay updated with our latest initiatives and innovations by following us on our social media channels! 

Together, let’s bridge the gap between AI and Islamic scholarship.

In [ ]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>